In [1]:
# import/parse the output file

filename = "wifi-cw-trace.out"



In [2]:
import re
from collections import defaultdict
from typing import Dict, List, Tuple, Optional

line_rx = re.compile(
    r"""
    \bnSTAs=(?P<nstas>\d+)\s*;
    \s*time=(?P<time>[0-9]*\.?[0-9]+)\s*;
    \s*node=(?P<node>\d+)\s*;
    \s*newCw=(?P<cw>\d+)\s*;
    """,
    re.VERBOSE,
)

def parse_trace(filename: str, target_node: int = 0) -> Dict[int, List[Tuple[float, int]]]:
    """
    Returns: dict { nSTAs -> [(time, cw), ...] } for the specified node.
    Lines that don't match or don't belong to target_node are ignored.
    """
    by_iter: Dict[int, List[Tuple[float, int]]] = defaultdict(list)
    with open(filename, "r", encoding="utf-8") as f:
        for raw in f:
            m = line_rx.search(raw)
            if not m:
                continue
            node = int(m.group("node"))
            if node != target_node:
                continue
            nstas = int(m.group("nstas"))
            t = float(m.group("time"))
            cw = int(m.group("cw"))
            by_iter[nstas].append((t, cw))

    # Ensure each iteration is time-ordered
    for k in by_iter:
        by_iter[k].sort(key=lambda x: x[0])
    return by_iter

def time_weighted_avg_cw_from_t3(
    samples: List[Tuple[float, int]],
    t0: float = 3.0,
) -> Optional[float]:
    """
    Implements the instructions:
      1) Consider only the traces for the iteration and start counting at t0 (default 3s).
      2) totalDuration = last_timestamp - max(t0, first_timestamp_that_affects_t0)
      3-4) Sum CW[j] * (timestamp[j+1] - max(timestamp[j], t0)) for all j except the last sample
      5) Return (sum weightedCW) / totalDuration
    Returns None if duration <= 0 or insufficient data.
    """
    if not samples:
        return None

    times = [t for t, _ in samples]
    cws   = [cw for _, cw in samples]

    # If all events happen before t0 or we have only one event after trimming, handle carefully
    # Find the index of the last update strictly before t0 (to know which CW is in effect at t0)
    import bisect
    idx = bisect.bisect_right(times, t0) - 1

    # Build a working view that starts at t0:
    work_times: List[float] = []
    work_cws:   List[int]   = []

    if idx >= 0:
        # The CW at t0 is cws[idx], effective from t0 until times[idx+1] (if any)
        work_times.append(t0)
        work_cws.append(cws[idx])
        # Then include all updates that occur AFTER t0
        start = idx + 1
    else:
        # No update before t0; if the first event is after t0, we start at that first event time
        # (we can't know which CW was in effect before the first event)
        start = 0
        if times[0] < t0:
            # Shouldn't happen given idx logic, but guard anyway
            t0 = times[0]
        work_times.append(max(t0, times[0]))
        work_cws.append(cws[0])
        start = 1  # subsequent events

    # Append remaining events as they are
    for j in range(start, len(times)):
        work_times.append(times[j])
        work_cws.append(cws[j])

    # If all constructed timestamps are identical or we have only one point, duration is zero/unknown
    if len(work_times) < 2:
        return None

    # Total duration is (last - first) per instructions
    total_duration = work_times[-1] - work_times[0]
    if total_duration <= 0:
        return None

    # Sum weighted CW for all *but the last* (since its end time is unknown)
    weighted_sum = 0.0
    for j in range(len(work_times) - 1):
        dt = work_times[j + 1] - work_times[j]
        if dt > 0:
            weighted_sum += work_cws[j] * dt

    return weighted_sum / total_duration if total_duration > 0 else None

def compute_time_avg_cw_per_iteration(filename: str, t0: float = 3.0) -> Dict[int, Optional[float]]:
    """
    Reads the file, groups by iteration (unique nSTAs), and returns:
        { nSTAs : time_weighted_avg_cw_from_t3(...) }
    """
    grouped = parse_trace(filename, target_node=0)
    results: Dict[int, Optional[float]] = {}
    for nstas, samples in grouped.items():
        avg = time_weighted_avg_cw_from_t3(samples, t0=t0)
        results[nstas] = avg
    return results

# ---------- Example usage ----------
if __name__ == "__main__":
    filename = "wifi-cw-trace.out"  # <- set/keep this variable as requested
    results = compute_time_avg_cw_per_iteration(filename, t0=3.0)
    # Pretty print
    for nstas in sorted(results):
        avg = results[nstas]
        print(f"nSTAs={nstas}: time-avg CW (t>=3s, node 0) = {avg if avg is not None else 'N/A'}")


nSTAs=5: time-avg CW (t>=3s, node 0) = 65.80347729076767
nSTAs=10: time-avg CW (t>=3s, node 0) = 186.06199246461784
nSTAs=15: time-avg CW (t>=3s, node 0) = 213.06391366478465
nSTAs=20: time-avg CW (t>=3s, node 0) = 378.7812882697779
nSTAs=25: time-avg CW (t>=3s, node 0) = 327.1829217002119
nSTAs=30: time-avg CW (t>=3s, node 0) = 408.85382923997474
nSTAs=35: time-avg CW (t>=3s, node 0) = 427.9287693976808
nSTAs=40: time-avg CW (t>=3s, node 0) = 431.1101608016173
